---
# <center> Лабораторна робота №9 </center>
## __Тема. Алгоритми на рядках__
## __Мета:__ освоїти низку основних алгоритмів на рядках засобами мови Python.
### _Викoнав Міняйло Марк 25.05.24_
-----

## Завданя для самостійної роботи
__1 ) Повторити код, наведений вище для довільних рядків__
#### Наївний пошук шаблонів (The Naive String Matching Algorithm)



In [4]:
def naive_match(p, t):
    assert len(p) <= len(t)  # припускаємо, що довжина шаблону не більша за довжину тексту
    occurrences = []
    for i in range(0, len(t)-len(p)+1):  # для кожного вирівнювання шаблону на текст
        match = True  # припускаємо, що є збіг, доки не доведено зворотнє
        for j in range(0, len(p)):  # для кожної позиції шаблону
            if t[i+j] != p[j]:
                match = False  # принаймні один символ не збігається
                break
        if match:
            occurrences.append(i)
    return occurrences
    
    #012345678
t = 'abacadaba'
p = 'aba'
naive_match(p, t)

[0, 6]

#### Z-функція

In [6]:
def z_func(s):
    # Ініціалізуємо масив Z, в якому будемо зберігати значення Z-функції
    Z = [len(s)] + [0] * len(s)  # Встановлюємо довжину рядка s у першому елементі масиву Z, а решту заповнюємо нулями
    assert len(s) > 1  # Перевіряємо, чи довжина рядка s більше 1
    
    # Початкова порівняння s[1:] з префіксом
    for i in range(1, len(s)):
        # Порівнюємо символи s[i] та s[i-1]. Якщо вони однакові, збільшуємо значення Z[1]
        if s[i] == s[i - 1]:
            Z[1] += 1
        else:
            break  # Якщо знайдений символ не збігається з попереднім, припиняємо процес
    
    # Установлюємо значення змінних r і l
    r, l = 0, 0
    if Z[1] > 0:
        r, l = Z[1], 1
    
    # Обчислення Z-функції для кожного k від 2 до довжини рядка s
    for k in range(2, len(s)):
        assert Z[k] == 0  # Перевірка, що значення Z[k] спочатку дорівнює 0
        if k > r:
            # Випадок 1: k за межами поточного діапазону [l, r]
            for i in range(k, len(s)):
                # Порівнюємо символи s[i] та s[i-k]. Якщо вони збігаються, збільшуємо значення Z[k]
                if s[i] == s[i - k]:
                    Z[k] += 1
                else:
                    break
            # Оновлюємо значення змінних r та l
            r, l = k + Z[k] - 1, k
        else:
            # Випадок 2: k знаходиться всередині поточного діапазону [l, r]
            # Обчислюємо довжину бета (nbeta) та значення Z-функції на попередньому кроці (Zkp)
            nbeta = r - k + 1
            Zkp = Z[k - l]
            if nbeta > Zkp:
                # Підвипадок 2а: Zkp виграє
                Z[k] = Zkp
            else:
                # Підвипадок 2б: Порівнюємо символи, що знаходяться безпосередньо після r
                nmatch = 0
                for i in range(r + 1, len(s)):
                    if s[i] == s[i - k]:
                        nmatch += 1
                    else:
                        break
                l, r = k, r + nmatch
                Z[k] = r - k + 1
    return Z

       #123456789
z_func('antaranto')

[9, 0, 0, 1, 0, 3, 0, 0, 0, 0]

#### Пошук підрядка в рядку

In [28]:
def zMatch(p, t):
    # Формуємо розширений рядок s, де p – це шаблон, а t – текст, який шукаємо.
    s = p + "$" + t
    
    # Обчислюємо масив Z для рядка s.
    Z = z_func(s)
    
    # Ініціалізуємо список occurrences для зберігання позицій, де відбувається збіг з шаблоном.
    occurrences = []
    
    # Проходимо по всіх позиціях збігу шаблону з текстом у рядку s.
    # Починаємо з позиції, що відповідає кінцю шаблону (len(p) + 1) і до кінця рядка s.
    for i in range(len(p) + 1, len(s)):
        # Якщо значення Z на поточній позиції рівне довжині шаблону, то це означає, що відбувся збіг.
        if Z[i] == len(p):
            # Додаємо позицію, де відбувся збіг, до списку occurrences.
            # Позиція розраховується як i – (len(p) + 1), оскільки ми розглядаємо розширений рядок s,
            # де шаблон p розташований на початку, перед "$".
            occurrences.append(i - (len(p) + 1))
    
    # Повертаємо список знайдених збігів.
    return occurrences

t = "анан$антарктида анаконда анчоус ананас"
calculated_z = z_func(t)
print("[ " + ", ".join(list(t)) + "   ]")
print(calculated_z)

[ а, н, а, н, $, а, н, т, а, р, к, т, и, д, а,  , а, н, а, к, о, н, д, а,  , а, н, ч, о, у, с,  , а, н, а, н, а, с   ]
[38, 0, 2, 0, 0, 2, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 3, 0, 1, 0, 0, 0, 0, 1, 0, 2, 0, 0, 0, 0, 0, 0, 4, 0, 3, 0, 1, 0, 0]


#### Стиснення рядка

In [37]:
def compress_with_z(s):
    # Обчислюємо Z-вектор для рядка s
    z_vec = z_func(s)
    
    # Проходимо по елементах Z-вектора, починаючи з індексу 1
    for i in range(1, len(s)):
        # Перевіряємо, чи виконуються умови для оптимального стиснення
        if (i + z_vec[i] == len(s)) and (z_vec[i] % i == 0):
            # Якщо умови виконуються, повертаємо частину рядка s, яка буде стиснута
            return s[:i]
    
    # Якщо жодна пара (i, z_vec[i]) не відповідає умовам, повертаємо вихідний рядок s без змін
    else:
        return s

s = "водалава водалава "
print(z_func(s))
print(compress_with_z(s))

[18, 0, 0, 0, 0, 0, 1, 0, 0, 9, 0, 0, 0, 0, 0, 1, 0, 0, 0]
водалава 


__2 ) Пояснити асимптотику наївного алгоритму пошуку підрядка__
   
Алгоритмічна складність в найгіршому випадку наївного алгоритму = $O(|t||p|)$.

Алгоритм має 2 цикли: зовнішній (for i in range(0, len(t) - len(p) + 1)) і внутрішній(for j in range(0, len(p)))

Кількість ітерацій першого циклу = t - p + 1, що дорівнює O(t)

Кількість ітерацій другого = O(p)

Загальна кількість операцій дорівнює добутку кількості ітерацій обох циклів, що дорівнює O(tp)

А модулі означають що число не може бути відємне


__3 ) Обчислити асимптотичну складність алгоритму стиснення рядка за допомогою z-функції__

compress_with_z, має дві вагомі фрагменти коду, які складають його асимптотичну складність

Перший це z_vec = z_func(s) - Це стандартний алгоритм який має O(s) складність (бо має два зовнішні цикли for від s)

Другий це зовнішній цикл for i in range(1, len(s)) - він прогодить по масиву S - тим самим даючи O(s) складність

Комбінуючи всі частини, загальна асимптотична складність алгоритму compress_with_z становить O (s + s) що = O(s)

---
#  <center>  Контрольні запитання </center>
1. _Що таке «префікс-функція» у контексті алгоритмів на рядках, і як вона відрізняється від Z-функції?_
   
Префікс-функція для рядка s довжиною n визначається як масив π довжиною n. Z-функція для рядка s довжиною n визначається як масив довжиною n.
Основна відмінність між префікс-функцією та Z-функцією полягає в тому, що префікс-функція шукає найбільший префікс, який є суфіксом підрядка, тоді як Z-функція шукає найбільший префікс, який є префіксом суфікса

2. _Що таке Z-функція у вищому контексті алгоритмів на рядках, і яке вона має значення для розв’язання задач?_

Z-функція використовується в різних алгоритмах обробки рядків, включаючи пошук паттернів, пошук періодичності, пошук префіксних функцій та інше. Вона є ефективним інструментом для розв’язання цих задач, оскільки дозволяє виконувати ці операції за лінійний час

3. _Які існують підходи до розв’язання задачі «найдовший спільний підрядок» для двох рядків?_

Існують різноманітні алгоритми, такі як наївний пошук шаблонів, Z-функція, пошук підрядка в рядку, стиснення рядку.
   
4. _Як можна застосувати алгоритми на рядках у задачах обробки природної мови або обробки текстів?_

   Можливо шукати потрібний текст, або перевіряти текст на плагіат


---